<a href="https://colab.research.google.com/github/robertue1/News_topic_modeling_using_NLP/blob/main/DEProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import urllib3
from urllib3 import request
import json
from pprint import pprint
import pandas as pd
import certifi
import numpy as np
import time
import re

#other needed tools
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, Float, Date
from collections import defaultdict

import gensim
from nltk.stem import WordNetLemmatizer
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
import nltk
nltk.download('wordnet')

http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# I'll be fetching the whole set of articles in one call,
#also, this time I'll be adding the word Chavez and Maduro as another possible match for news. 


In [ ]:
#From 1999 to 2021
API_KEY = '189604c2-ac97-4295-9ccf-e3556505cff3'
query = "Venezuela | Chavez | Maduro"
query_fields = "webTitle"
section = "news"
show_fields = "body"
page_size = "100"
from_date = "1999-01-01"
to_date = "2021-12-01"
retrieved_articles = 0
for i in range(1,100):
  query_url = f"http://content.guardianapis.com/search?" \
              f"api-key={API_KEY}" \
              f"&q={query}" \
              f"&query-fields ={query_fields}"\
              f"&page={i}" f"&page-size={page_size}"\
              f"&show-fields={show_fields}"\
              f"&from-date={from_date}" \
              f"&to-date={to_date}"\

  r = http.request('GET', query_url)
  time.sleep(1)
  data = json.loads(r.data.decode('utf-8')) #Saving the articles in data
  num_articles = data['response']['total'] #This is the total number of articles in the range of search. 
  #Here, I'm creating 'df' if it is the first time running the code, otherwise, I'll be concatenating the new data to the exisiting 'df'. 
  if i == 1:
    df99_21 = pd.json_normalize(data['response'],record_path=['results'])
  else:
    df99_21 = pd.concat([df99_21,pd.json_normalize(data['response'],record_path=['results'])], axis=0)
  retrieved_articles = int(page_size)*i
  #Now, I'm gonna compare the number of articles retrieved versus the total of 
  #articles, to see if i should break the for loop prematurely
  if retrieved_articles > num_articles:
    break
print (num_articles)

7636


In [ ]:
df99_21.sample(2)

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName,fields.body
46,lifeandstyle/2016/nov/19/painting-train-hare-b...,article,lifeandstyle,Life and style,2016-11-19T07:00:01Z,Which painting depicts a train bearing down on...,https://www.theguardian.com/lifeandstyle/2016/...,https://content.guardianapis.com/lifeandstyle/...,False,pillar/lifestyle,Lifestyle,<h2>The questions</h2> <p><strong>1 </strong>W...
2,commentisfree/2019/feb/18/split-labour-writers...,article,commentisfree,Opinion,2019-02-18T12:04:32Z,Are the gang of seven right to split from Labo...,https://www.theguardian.com/commentisfree/2019...,https://content.guardianapis.com/commentisfree...,False,pillar/opinion,Opinion,"<h2><a href=""https://www.theguardian.com/profi..."


In [ ]:
#As there are a lot of articles that mention Venezuela too briefly, I'll focus just on those that include Venezuela, Chavez or Maduro in the title.
dfve = df99_21[df99_21['webTitle'].str.contains("Venezuela|Chavez|Maduro")]
dfve.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122 entries, 0 to 52
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1122 non-null   object
 1   type                1122 non-null   object
 2   sectionId           1122 non-null   object
 3   sectionName         1122 non-null   object
 4   webPublicationDate  1122 non-null   object
 5   webTitle            1122 non-null   object
 6   webUrl              1122 non-null   object
 7   apiUrl              1122 non-null   object
 8   isHosted            1122 non-null   bool  
 9   pillarId            1108 non-null   object
 10  pillarName          1108 non-null   object
 11  fields.body         1122 non-null   object
dtypes: bool(1), object(11)
memory usage: 106.3+ KB


In [ ]:
dfve.dropna(subset=['fields.body'], inplace=True)
dfve.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122 entries, 0 to 52
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1122 non-null   object
 1   type                1122 non-null   object
 2   sectionId           1122 non-null   object
 3   sectionName         1122 non-null   object
 4   webPublicationDate  1122 non-null   object
 5   webTitle            1122 non-null   object
 6   webUrl              1122 non-null   object
 7   apiUrl              1122 non-null   object
 8   isHosted            1122 non-null   bool  
 9   pillarId            1108 non-null   object
 10  pillarName          1108 non-null   object
 11  fields.body         1122 non-null   object
dtypes: bool(1), object(11)
memory usage: 106.3+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def cleaning_content(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    #Removing accents
    text = re.sub(u"[àáâãäå]", 'a', text)
    text = re.sub(u"[èéêë]", 'e', text)
    text = re.sub(u"[ìíîï]", 'i', text)
    text = re.sub(u"[òóôõö]", 'o', text)
    text = re.sub(u"[ùúûü]", 'u', text)
    text = re.sub(u"[ýÿ]", 'y', text)
    text = re.sub(u"[ß]", 'ss', text)
    text = re.sub(u"[ñ]", 'n', text)
    text = text.replace("\\", " ")
    return re.sub(clean, '', text)


dfve['clean'] = dfve['fields.body'].apply(cleaning_content)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#Renaming the columns for convenience
dfve.rename({'webPublicationDate':'date',\
                'webTitle':'title',\
                'webUrl':'url',\
                'clean':'content'}, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#Comparing the text before and after applying the 'Cleaning_content' function, which is used to eliminate links, html tags and accents. 

In [ ]:
dfve.iloc[0]['fields.body']

'<p> Venezuela’s government is halting negotiations with its opponents in retaliation for the extradition to the US of a close ally of president Nicolás Maduro, who prosecutors believe could be the most significant witness ever about corruption in the South American country.</p> <p>Jorge Rodríguez, who has been heading the government’s delegation, said his team wouldn’t travel to Mexico City for the next scheduled round of negotiations.</p> <p>The announcement capped a tumultuous day that saw Colombian-born businessman Alex Saab placed on a US-bound plane in Cape Verde after a 16-month fight by Maduro and his allies, including Russia, who consider Saab a Venezuelan diplomat.</p> <aside class="element element-rich-link element--thumbnail"> <p> <span>Related: </span><a href="https://www.theguardian.com/us-news/2021/jul/08/venezuela-caracas-gun-battles-police-gangs">Venezuela: intense gun battles rage in Caracas between gangs and police</a> </p> </aside>  <p>The Venezuelan government in S

In [ ]:
dfve.iloc[0]['content']

' Venezuela’s government is halting negotiations with its opponents in retaliation for the extradition to the US of a close ally of president Nicolas Maduro, who prosecutors believe could be the most significant witness ever about corruption in the South American country. Jorge Rodriguez, who has been heading the government’s delegation, said his team wouldn’t travel to Mexico City for the next scheduled round of negotiations. The announcement capped a tumultuous day that saw Colombian-born businessman Alex Saab placed on a US-bound plane in Cape Verde after a 16-month fight by Maduro and his allies, including Russia, who consider Saab a Venezuelan diplomat.   Related: Venezuela: intense gun battles rage in Caracas between gangs and police    The Venezuelan government in September named Saab – who was arrested in June 2020 when his plane stopped in Cape Verde to refuel – as a member of its negotiating team in talks with the opposition in Mexico, where the two sides are looking to solve

In [ ]:
#Creating a New Database with this 1122 news articles
engine = create_engine('sqlite:///newsve.db', echo=False)
meta = MetaData()
newstable = Table (
    'news_table', meta,
    Column('id', String, primary_key = True),
    Column('date', Date),
    Column('title', String),
    Column('url', String),
    Column('content', String) 
)
#Selecting just the columns I consider important from the dfve.
dfve[['id', 'date', 'title', 'url', 'content']].to_sql('news_table', con=engine, if_exists='append')

In [ ]:
engine.execute("SELECT * FROM news_table limit 5").fetchall()

[(0, 'world/2021/oct/17/brutal-aggression-venezuela-halts-talks-with-opposition-after-envoy-extradited-to-us', '2021-10-17T03:31:01Z', '‘Brutal aggression’: Venezuela halts talks with opposition after envoy extradited to US', 'https://www.theguardian.com/world/2021/oct/17/brutal-aggression-venezuela-halts-talks-with-opposition-after-envoy-extradited-to-us', ' Venezuela’s government is halting negotiations with its opponents in retaliation for the extradition to the US of a close ally of president Nicolas  ... (3526 characters truncated) ... ations and the attorney general’s office did not immediately respond to requests for comment. Associated Press and Reuters contributed to this report'),
 (1, 'world/2020/sep/16/venezuela-un-report-crimes-against-humanity-maduro-government', '2020-09-16T14:41:29Z', 'Venezuela: UN accuses Maduro government of crimes against humanity', 'https://www.theguardian.com/world/2020/sep/16/venezuela-un-report-crimes-against-humanity-maduro-government', 'Venezu

In [ ]:
#Just selecting columns of interest from now on, and calling it df
df = dfve[['id', 'date', 'title', 'url', 'content']]

In [ ]:
#Let's try to focus on building a topic modeling tool for the years between 1999 and 2004
start_date = '1999-01-01'
end_date = '2004-01-01'
mask99_03 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
df[mask99_03].sample(5)

,id,date,title,url,content
0,stage/2003/mar/14/theatre.artsfeatures2,2003-03-14T11:36:11Z,"Venezuela, Arcola, London",https://www.theguardian.com/stage/2003/mar/14/...,The excellent Arcola theatre proves its worth ...
10,world/2003/aug/10/venezuela.theobserver,2003-08-09T23:57:58Z,Venezuela's slum army takes over,https://www.theguardian.com/world/2003/aug/10/...,"The economy is in a shambles, the country is t..."
11,world/2000/aug/10/iraq.brianwhitaker,2000-08-10T00:05:51Z,US furious at Venezuelan leader's visit to Saddam,https://www.theguardian.com/world/2000/aug/10/...,"The Venezuelan president, Hugo Chavez, is due ..."
81,media/2003/jan/08/broadcasting,2003-01-08T08:17:19Z,How talk show hosts used canned Castro to fool...,https://www.theguardian.com/media/2003/jan/08/...,Two Miami radio presenters used a tape recordi...
78,world/2002/apr/08/venezuela.oil,2002-04-08T13:49:36Z,Venezuelan president purges oil company,https://www.theguardian.com/world/2002/apr/08/...,"Venezuela's president, Hugo Chavez, sacked sev..."


In [ ]:
 def tokenize(text):
        """Returns tokenized representation of words in lemma form excluding stopwords"""
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS \
                    and len(token) > 2:  # drops words with less than 3 characters
                result.append(lemmatize(token))
        return result
def lemmatize(token):
        """Returns lemmatization of a token"""
        return WordNetLemmatizer().lemmatize(token, pos='v')
        
def string_joiner(x):
        return ' '.join([str(item) for item in x])  
    
#Applying lemmatization
df['lemma'] = df.content.apply(tokenize)
df['joined_lemma'] = df.lemma.apply(string_joiner)



In [ ]:
df.sample(5)

,id,date,title,url,content,lemma,joined_lemma
62,world/2017/apr/24/venezuela-protests-roads-blo...,2017-04-25T00:10:54Z,Venezuela anti-government protesters paralyse ...,https://www.theguardian.com/world/2017/apr/24/...,Venezuela’s major transport arteries juddered ...,"[venezuela, major, transport, arteries, judder...",venezuela major transport arteries judder halt...
68,world/2019/apr/15/china-mike-pompeo-venezuela-...,2019-04-15T15:55:45Z,‘Pompeo has lost his mind’: China denies US cl...,https://www.theguardian.com/world/2019/apr/15/...,China has rejected US claims that it is partly...,"[china, reject, claim, partly, blame, disinteg...",china reject claim partly blame disintegration...
61,world/2018/sep/08/trump-nicolas-maduro-overthr...,2018-09-08T16:32:15Z,Trump officials met with Venezuelan officers t...,https://www.theguardian.com/world/2018/sep/08/...,The Trump administration held meetings with di...,"[trump, administration, hold, meet, dissident,...",trump administration hold meet dissident venez...
53,world/2017/aug/08/venezuela-hugo-chavez-ruben-...,2017-08-08T09:00:34Z,Chávez loyalists hold firm amid chaos in Venez...,https://www.theguardian.com/world/2017/aug/08/...,Ruben Ávila – an idealist raised on Marxist th...,"[ruben, ávila, idealist, raise, marxist, theor...",ruben ávila idealist raise marxist theory ster...
21,world/us-embassy-cables-documents/229894,2010-12-09T13:11:19Z,US embassy cables: Safety concerns at Venezuel...,https://www.theguardian.com/world/us-embassy-c...,"Thursday, 15 October 2009, 21:59C O N F I D E ...","[thursday, october, section, caracas, sipdis, ...",thursday october section caracas sipdis energy...


In [ ]:
#Saving the df as a CSV and calling it a day.

In [ ]:
df.to_csv('data.csv')

In [ ]:
#Reading the CSV and getting it into a DF. 
testing = pd.read_csv('/data.csv', index_col=0)


In [ ]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122 entries, 0 to 52
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1122 non-null   object
 1   date          1122 non-null   object
 2   title         1122 non-null   object
 3   url           1122 non-null   object
 4   content       1072 non-null   object
 5   lemma         1122 non-null   object
 6   joined_lemma  1071 non-null   object
dtypes: object(7)
memory usage: 70.1+ KB


In [ ]:
testing.dropna(subset= ['content', 'joined_lemma'],inplace=True, axis=0)

In [ ]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1071 entries, 0 to 52
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1071 non-null   object
 1   date          1071 non-null   object
 2   title         1071 non-null   object
 3   url           1071 non-null   object
 4   content       1071 non-null   object
 5   lemma         1071 non-null   object
 6   joined_lemma  1071 non-null   object
dtypes: object(7)
memory usage: 66.9+ KB


In [ ]:
df = testing.copy()

In [ ]:
#Topic Modeling for the years between 1999 and 2004
start_date = '1999-01-01'
end_date = '2004-01-01'
mask99_03 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
#For the years [99-2004)

#Stopwords
words_to_add = ['venezuela', 'chavez', 'caracas', 'say',\
                'miami', 'time', 'live', 'self', 'style',\
                'yellow', 'brazil','theguardian', 'weeks', 'ago', 'year', 'world', 'football']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=0.021, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask99_03].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
#Now, applying the NMF
 
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-4:]]])))



Topic 1: oil company,president hugo,general strike,state oil
Topic 2: peaceful revolution,constituent assembly,latin america,new constitution
Topic 3: fidel castro,april fool,castro want,anti castro
Topic 4: qualify finals,beat uruguay,play australia,south american
Topic 5: pedro carmona,oil price,white house,unite state


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
#Finding the links for some of the words in the topics. 
df[(df.title.str.contains('general | strike', flags=re.IGNORECASE, regex=True)) & (mask99_03)][['title','url']]

,title,url
74,Chavez turns to troops amid strike chaos,https://www.theguardian.com/world/2003/jan/18/venezuela
76,Venezuela arrests strike leader,https://www.theguardian.com/world/2003/feb/21/venezuela
77,Venezuela strike leader 'arrested',https://www.theguardian.com/world/2003/feb/20/venezuela1
8,Chavez tries to steady the ship as strike peters out,https://www.theguardian.com/world/2003/feb/07/venezuela.duncancampbell
33,Bank strike deepens crisis in Venezuela,https://www.theguardian.com/world/2003/jan/09/venezuela.duncancampbell
37,Venezuelan strike halts oil exports,https://www.theguardian.com/world/2002/dec/07/oil.venezuela
38,Venezuelan president defies calls for general election,https://www.theguardian.com/world/2002/dec/16/venezuela
27,Oil exports slump as Venezuelan strike drags on,https://www.theguardian.com/world/2002/dec/11/venezuela


In [ ]:
#Finding the links for some of the words in the topics. 
df[(df.title.str.contains('finals', flags=re.IGNORECASE, regex=True)) & (mask99_03)][['title','url']]

,title,url


In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [ ]:
#Topic Modeling for the years between 2004 and 2009)
start_date = '2004-01-01'
end_date = '2009-01-01'
mask04_08 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
#For the years [2004-2009)

#Stopwords
words_to_add = ['venezuela', 'chavez', 'caracas', 'say', 'time', 'live', 'self', 'style', 'theguardian', 'weeks', 'ago', 'year']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=.03, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask04_08].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-4:]]])))


Topic 1: latin american,george bush,bush administration,venezuelan president
Topic 2: land reform,radio television,press freedom,president hugo
Topic 3: human right,latin american,simon bolivar,latin america
Topic 4: public service,socialist revolution,term limit,opinion poll
Topic 5: colombia farc,revolutionary arm,force colombia,arm force


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
#Finding the links for some of the words in the topics. 
df[(df.title.str.contains('socialist', flags=re.IGNORECASE, regex=True)) & (mask04_08)][['title','url']]

,title,url
48,Ben Whitford: Voters in Venezuela's urban areas abandon Hugo Chávez's socialist project,https://www.theguardian.com/commentisfree/cifamerica/2008/nov/24/venezuela-elections-hugo-chavez
7,"Venezuela and Iran: The Holocaust denier, the radical socialist, and their axis of unity",https://www.theguardian.com/world/2007/jul/25/venezuela.iran
70,Flares and medallions live again on Chavez's socialist TV,https://www.theguardian.com/media/2007/jun/09/venezuela.broadcasting
34,"Chavez is a populist, not a socialist",https://www.theguardian.com/commentisfree/2006/may/15/chavezispopulistnotasocia


In [ ]:
#From 2009 to 2014)
start_date = '2009-01-01'
end_date = '2014-01-01'
mask09_14 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
#For the years [2009-2014)

#Stopwords
words_to_add = ['venezuela', 'caracas', 'say', 'time', 'live',\
                'getty', 'venezuelan', 'play', 'nbsp', 'bst', 'min', 'look', 'year', 'ago', 'afp', 'simon', 'chavez']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
  
vectorizer = TfidfVectorizer(max_features=2000, min_df=0.10, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask09_14].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-5:]]])))


Topic 1: president nicolas,rule party,henrique capriles,national assembly,nicolas maduro
Topic 2: latin american,unite state,south america,socialist revolution,president hugo
Topic 3: henrique capriles,south america,unite state,latin american,latin america
Topic 4: socialist revolution,vice president,national assembly,human right,national guard
Topic 5: national assembly,south america,socialist revolution,oil revenues,oil company


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
#Finding the links for some of the words in the topics. 
df[(df.title.str.contains('socialist', flags=re.IGNORECASE, regex=True)) & (mask09_14)][['title','url']]

,title,url
57,Hugo Chávez vows to keep Venezuela 'on socialist path' after re-election,https://www.theguardian.com/world/2012/oct/08/venezuela-election-hugo-chavez


In [ ]:
#From 2014 to 2017)
start_date = '2014-01-01'
end_date = '2017-01-01'
mask14_17 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
#For the years [2014-2017)

#Stopwords
words_to_add = ['venezuela', 'caracas', 'say', 'time', 'live',\
                'getty', 'venezuelan', 'play', 'nbsp', 'bst', 'min', 'look', 'year', 'ago', 'afp']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=0.05, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask14_17].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-4:]]])))

Topic 1: nicolas maduro,recall referendum,oil price,national assembly
Topic 2: human right abuse,right abuse,latin america,human right
Topic 3: late president hugo,late president,president hugo,president hugo chavez
Topic 4: price control,currency control,exchange rate,black market
Topic 5: leader leopoldo,leader leopoldo lopez,opposition leader,leopoldo lopez


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
df[(df.title.str.contains('black market', flags=re.IGNORECASE, regex=True)) & (mask14_17)][['title','url']]

,title,url
67,Price controls and scarcity force Venezuelans to turn to the black market for milk and toilet paper,https://www.theguardian.com/global-development-professionals-network/2015/apr/16/venezuela-economy-black-market-milk-and-toilet-paper


In [ ]:
#From 2017 to 2021)
start_date = '2017-01-01'
end_date = '2021-01-01'
mask17_21 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}') 

In [ ]:
#For the years [2017-2021)

#Stopwords
words_to_add = ['venezuela', 'caracas', 'say', 'time', 'live', 'play', 'nbsp']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=0.01, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask17_21].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-4:]]])))

Topic 1: nicolas maduro,opposition leader,donald trump,juan guaido
Topic 2: work class family,human right,sanction maduro,oil price
Topic 3: import food medicine,electoral boycott,import food,black market
Topic 4: brief deliver thousands,sign morning,sign morning brief,morning brief
Topic 5: supreme court,security force,human right,constituent assembly


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
#Finding the links for some of the words in the topics. 
df[(df.title.str.contains('human', flags=re.IGNORECASE, regex=True)) & (mask17_21)][['title','url']]

,title,url
1,Venezuela: UN accuses Maduro government of crimes against humanity,https://www.theguardian.com/world/2020/sep/16/venezuela-un-report-crimes-against-humanity-maduro-government
21,"Venezuela’s Covid-19 death toll claims ‘not credible’, human rights group says",https://www.theguardian.com/world/2020/may/26/venezuela-coronavirus-death-toll-human-rights-watch
24,Venezuela wins UN human rights council seat despite record of abuses,https://www.theguardian.com/world/2019/oct/17/venezuela-un-human-rights-council-activists-outraged
81,'It's like 1984': Venezuela targets human rights defenders,https://www.theguardian.com/global-development/2019/jan/03/venezuela-human-rights-arrests-nicolas-maduro
23,Venezuela’s opposition wins EU’s prestigious human rights prize,https://www.theguardian.com/world/2017/dec/13/venezuelas-opposition-wins-eus-prestigious-human-rights-prize
28,Nine ways you can support Venezuelan human rights protests | Reynaldo Trombetta,https://www.theguardian.com/commentisfree/2017/may/10/support-venezuelan-human-rights-protests-maduro


In [ ]:
#Last piece of code, news from January to december 2021

In [ ]:
start_date = '2021-01-01'
end_date = '2021-12-01'
mask21 = (df['date'] > f'{start_date}') & (df['date'] < f'{end_date}')

In [ ]:

#Stopwords
words_to_add = ['venezuela', 'caracas', 'say', 'time', 'live', 'play', 'chavez']
stop_w = ENGLISH_STOP_WORDS.union(words_to_add)
 
vectorizer = TfidfVectorizer(max_features=2000, min_df=0.2, stop_words=stop_w, ngram_range=(2,3))
X = vectorizer.fit_transform(df[mask21].joined_lemma)
words = np.array(vectorizer.get_feature_names_out())
nmf = NMF(n_components=5, solver="cd")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-4:]]])))

Topic 1: venezuelan refugees,human right,latin american,latin america
Topic 2: refugees migrants,drug traffic,tell guardian,year old
Topic 3: president nicolas,president nicolas maduro,drug traffic,nicolas maduro
Topic 4: president nicolas maduro,president nicolas,nicolas maduro,political crisis
Topic 5: economic social,refugee agency,venezuelan migrants,million venezuelans


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [ ]:
#Done with this, the rest, will be the Streamlit App